In [55]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from utilities import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [51]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
N = len(y) # training set cardinality
D = tX.shape[1] # number of parameters ("dimensionality")

# Processing the data
Several observations are incomplete, as it can be shown looking at the number of "-999" in the dataset. 
To solve this issue, we can eliminate columns where too many data, say 70%, are missing, and we can impose the mean in the remaining cases.
Moreover, the values of the features are quite high, so it is a better practice to normalize the data before proceeding with the implementations.

In [52]:
print(np.count_nonzero(tX==-999)) # counting the missing data in tX

1580052


In [53]:
tX = missing_values(tX)
print(tX)

[[138.47        51.655       97.827      ...   0.           0.
  113.497     ]
 [160.937       68.768      103.235      ...   0.           0.
   46.226     ]
 [121.85852836 162.172      125.953      ...   0.           0.
   44.251     ]
 ...
 [105.457       60.526       75.839      ...   0.           0.
   41.992     ]
 [ 94.951       19.362       68.812      ...   0.           0.
    0.        ]
 [121.85852836  72.756       70.831      ...   0.           0.
    0.        ]]


In [64]:
tX = normalize(tX)

## Models 

Application of the 6 models expected for the project, on the prepocessed data.

## Linear Regression using Gradient Descent


In [59]:
max_iters = 500
gamma = 0.1
initial_w = np.zeros([D,])
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)

In [60]:
print(w)
print(loss)

[-1.31722092e-01 -6.73104679e-01 -5.75997620e-02  1.76021742e-02
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.44685316e-01
 -3.16366555e-02  1.75361413e-02 -4.63086299e-01  3.08609986e-01
  0.00000000e+00  1.65658562e-01 -4.01854088e-03 -5.63071156e-03
 -1.98018316e-01  5.09973314e-04  4.60167403e-03 -3.94879990e-02
  8.25067427e-03 -8.47846408e-02 -1.25008565e-01 -7.96018256e-02
 -1.22940432e-03  3.81088219e-03  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.02840434e-02]
0.40974372781389495


## Linear Regression usign Stochastic Gradient Descent

In [61]:
max_iters = 500
gamma = 0.01
batch_size = 1
initial_w = np.zeros([D,])
w, loss = least_squares_SGD(y, tX, initial_w, max_iters, gamma, batch_size)

In [62]:
print(w)
print(loss)

[-0.09159461 -0.11844551 -0.04814526 -0.01092648  0.          0.
  0.         -0.36202172 -0.00736941 -0.0584698  -0.09226844  0.32290135
  0.         -0.02306169  0.00171049  0.00570168 -0.07652171  0.05225835
  0.06709349 -0.01268062 -0.03078908 -0.08166884 -0.22016249 -0.06987739
 -0.11161883  0.06003848  0.          0.          0.         -0.02927074]
0.4235495794539486


## Least Squares Regression using Normal Equations

In [ ]:
for i in range(D):
    maximum = np.max(abs(tX[:,i]))
    if (maximum != 0.0) : X[:,i] = X[:,i]/maximum

w, loss = least_squares(y, tX)

LinAlgError: Singular matrix

In [ ]:
print(w)
print(loss)

## Ridge Regression using Normal Equations
We perform a cross-validation for chosing the optimal lambda, degree is fixed to 5. # Tuning also for the degree?

In [48]:
# Should we put the cross-validation template in a separate, external file?

def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    # get k'th subgroup in test, others in train:
    ind = k_indices[k,:]
    ind_tr = np.delete(k_indices, (k), axis = 0)
    x_tr = np.hstack(x[ind_tr])
    y_tr = np.hstack(y[ind_tr])
    # ridge regression:
    w = ridge_regression(y_tr, x_tr, lambda_)
    # calculate the loss for train and test data:
    e_tr = y_tr - basis_tr.dot(w)
    loss = 1/(2*len(y_tr)) * np.transpose(e_tr).dot(e_tr)
    
    return w, loss

seed = 1
degree = 5
k_fold = 3
lambdas = np.logspace(-4, 0, 30)
# split data in k fold
k_indices = build_k_indices(y, k_fold, seed)
for i in range(len(lambdas)):
        lambda_ = lambdas[i]
        tr_loss = 0
        for k in range(k_fold): 
            loss_tr= cross_validation(y, tX, k_indices, k, lambda_, degree)[1]
            tr_loss = tr_loss + loss_tr
        rmse_tr.append(math.sqrt(2 * tr_loss/k_fold))


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 166666 is different from 83333)

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)